# 1　基本構成

In [ ]:
from flask import Flask
from flask_restful import Resource, Api

`Resource` 用來處理 API 使用到的物件

In [ ]:
app = Flask(__name__)
api = Api(app)

創建一個 `Api` 物件並引入原先的 `Flask` 物件 

In [ ]:
class Student(Resource):
    def get(self, name):
        return {'student': name}

`Student` 類別會繼承(inheritance) `Resource` 類別


函式名稱直接命名為 HTTP 方法 get

In [ ]:
api.add_resource(Student, '/student/<string:name>')

app.run(port=5000)

將 resource 加入 API 當中，並指定 `Student` 類別及 endpoint 路徑

如此可在 Postman 當中以 GET http://127.0.0.1:5000/student/Rolf 存取

會回傳 `{"student": "Rolf"}`

註：使用 Flask-RESRTful 會自動轉換成 JSON，不用再使用 `jsonify`

***

# 2-1　商店 API 實作例 - GET、POST

In [ ]:
class Item(Resource):
    def get(self, name):
        item = next(filter(lambda x:x['name'] == name, items), None)
        return {'item': item}, 200 if item else 404

其中 `filter(lambda x: x['name'] == name, items)` 的意思同下列程式碼：

In [ ]:
for item in items:
    if item['name'] == name:
        return item

`lambda` 函式的功能是 x 輸入後將 `x['name'] == name` 的真偽值輸出

`filter` 函式的功能是將 `items` 此商品 list 中的元素丟入指定函式中，若結果為真則保留

如此，滿足 `item['name'] == name` 的元素(此處為單一商品 dict)會被保留下來

`next` 函式的作用為尋找下一個元素，若無則輸出預設值 `None`

亦可使用 `list(filter())` 來回傳所有過濾後的元素列表

而在 return 後面的 200 為 HTTP 狀態碼，常見的有：
- 200 OK (最常見)
- 201 CREATED
- 202 ACCPETED
- 400 BAD REQUEST
- 401 UNAUTHORIZED
- 403 FORBIDDEN
- 404 NOT FOUND
- 500 INTERNAL SERVER ERROR


其中 400 系列為使用者造成的錯誤、500 系列為伺服器本身的錯誤

In [ ]:
    def post(self, name):
        data = request.get_json()
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists."\
                    .format(name)}, 400

        item = {'name': name, 'price': data['price']}
        items.append(item)
        return item

註： `is not None` 可以直接省略

# 2-2　商店 API 實作例 - DELETE、PUT

In [ ]:
    def delete(self, name):
        global items
        items = list(filter(lambda x: x['name'] != name, items))
        return {'message': 'Item deleted'}

用 `filter` 函式將 `items` 過濾，只留下指定名稱之外的物件，達成刪除

如果不加 `global items` 指定使用全域的 `items`

Python 會認為使用的是 `delete` 函式中的 `items`

兩個 `items` 在同一行，會造成未定義就先使用的情況，產生錯誤

In [ ]:
    def put(self, name):
        data = request.get_json()
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            item = {'name': name, 'price': data['price']}
            items.append(item)
        else:
            item.update(data)
        return item

dict 有 `.update(data)` 函式可使用

註：PUT 具有冪等 (idempotent) 特性，意指無論執行幾次，最後都會得到相同結果

重複執行 PUT 指令不會額外創造出多個物件，但 POST 則有可能

***

# 3-1　驗證功能 - 設定

In [ ]:
from flask_jwt import JWT, jwt_required, current_identity

app = Flask(__name__)
app.secret_key = 'wolf'
api = Api(app)

JWT 代表 JSON Web Token

須設定 `secret_key` 以供 `Flask_jwt` 進行加密

為了執行驗證功能，需要另外兩個檔案：user.py 及 security.py

- user.py

In [ ]:
class User:
    def __init__(self, _id, username, password):
        self.id = _id
        self.username = username
        self.password = password

id 是 python keyword，故使用 `_id`

- security.py

In [ ]:
from user import User

users = [
    User(1, 'Starlord', 'infinitygauntlet')
]

username_mapping = {u.username: u for u in users}
userid_mapping = {u.id: u for u in users}


`{u.username: u for u in users}` 是 dict comprehension

如上 `username_mapping` 的值會是 `{'bob': User(1, 'bob', 'asdf')}` 、格式為 dict



In [ ]:
def authenticate(username, password):
    user = username_mapping.get(username, None)
    if user and user.password == password:
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_mapping.get(user_id, None)

security.py 需要包含 `authenticate` 及 `identity` 兩個函式

在這兩個函式中會使用 `dict.get(key, default)` 

以 `username` 及 `user_id` 作為鍵值 (key) 對上述兩個 mapping dict 進行搜尋

不直接使用 `['key']` 而使用 `.get()` 的原因是可以指定 找不到時 的預設回傳值

註：若使用 Python 2.7 或更早版本，不建議直接比對字串，建議使用如下方法：

In [ ]:
from werkzeung.security import safe_str_cmp

if user and safe_str_cmp(user.password, password):
    pass

# 3-2　驗證功能 - 實作

In [ ]:
from security import authenticate, identity
from flask_jwt import JWT, jwt_required, current_identity

jwt = JWT(app, authenticate, identity)

`jwt_required` 用以加入驗證用 decorator 到函式上

`JWT()` 會創造路徑為 /auth 的 endpoint

In [ ]:
@jwt_required()
    def get(self, name):
        pass

在函式上加入 `@jwt_required()` decorator 可讓此函式加上驗證保護

***

# 4　輸入資料修剪 (Parse)

In [ ]:
from flask_restful import Resource, Api, reqparse

def put(self, name):
    parser = reqparse.RequestParser()
    parser.add_argument('price',
        type=float,
        required=True,
        help="This field cannot be left blank!"
    )
    data = parser.parse_args()

`parser = reqparse.RequestParser()` 建立一個新的 parser

`parser.add_argument()` 加入 parser 修剪對象 (key) 及條件

`data = parser.parse_args()` 針對設定對象執行修剪

如此不論使用者傳入的 JSON 包含什麼資料，都會被修剪成只剩下指定的資料

註：此函式可用於修剪 HTML 的 Form

通常 parser 會定義在主類別之下，避免在函式中重複複製貼上

故可將程式碼修改如下：

In [ ]:
class Item(Resource):
    parser = reqparse.RequestParser()
    parser.add_argument('price',
        type=float,
        required=True,
        help="This field cannot be left blank!"
    )

    def post(self, name):
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists.".format(name)}

        data = Item.parser.parse_args()
        item = {'name': name, 'price': data['price']}
        items.append(item)
        return item

要注意的是，在 `post` 函式中的 `if` 條件式位於 `data = Item.parser.parse_args()` 前面

因為在判斷物品已經存在的情況下，不會新增物品，就沒有必要對資料作修剪